<a href="https://colab.research.google.com/github/Gayatri2002/MBTI-personality-prediction-using-NLP/blob/main/tarp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Complete Chat Analysis And Personality
Detection Using NLP**   TARP Project


In [ ]:
pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45228 sha256=30f97a1c6247874c76ccecd4fa9521c6cb406a434323422b1e2a04bb02d3a5e3
  Stored in directory: /root/.cache/pip/wheels/e8/9d/fd/0a594a1bc8d493935d319ac40d64d79f5b4117ef4d37b1b678
Successfully built ws4py


In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score
import pickle
import os.path
import plotly.offline as pyo
import plotly.graph_objs as go
import spacy
from nltk.stem.snowball import SnowballStemmer
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MBTI 500.csv')

In [ ]:
df.head()

,posts,type
0,know intj tool use interaction people excuse a...,INTJ
1,rap music ehh opp yeah know valid well know fa...,INTJ
2,preferably p hd low except wew lad video p min...,INTJ
3,drink like wish could drink red wine give head...,INTJ
4,space program ah bad deal meing freelance max ...,INTJ


In [ ]:
df['posts'][0]

'know intj tool use interaction people excuse antisocial truly enlighten mastermind know would count pet peeze something time matter people either whether group people mall never see best friend sit outside conversation jsut listen want interject sit formulate say wait inject argument thought find fascinate sit watch people talk people fascinate sit class watch different people find intrigue dad intj u stand look like line safeway watch people home talk people like think military job people voluntarily go job important show deference endanger live glorify way civilian think pretty ignorant general think military necessary defense mechanism political tactic feel like u specifically invest much money could put money education whatnot though personally sound budget aernative really comment one way base two politician eye year ago come name somewhat important kinda role model nowadays pick keep score individual level mean little vary accord number condition day may score high others low sw

In [ ]:
df['type'][0]

'INTJ'

In [ ]:
df['type'].unique()
TYPES=['ENFJ','ENFP','ENTJ','ENTP','ESFJ','ESFP','ESTJ','ESTP','INFJ','INFP','INTJ','INTP','ISFJ','ISFP','ISTJ','ISTP']

In [ ]:
print(f"Total of {len(df['type'].unique())} types of classified MBTI posts")

Total of 16 types of classified MBTI posts


In [ ]:
df.isnull().sum()

posts    0
type     0
dtype: int64

Data Visualization

In [ ]:
df_bar_chart=df.groupby('type').count()


trace1 = go.Bar(x=df_bar_chart.index, y=df_bar_chart['posts'])

data = [trace1]
layout = go.Layout(title='MBTI # Classified Posts per Type')

fig = go.Figure(data=data, layout=layout)

fig.show()

In [ ]:
filename = '/content/drive/MyDrive/Colab Notebooks/mbti_svm_v1.sav'

In [ ]:
recreate_model=True

In [ ]:
X = df['posts'] # features
y = df['type']  # labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Check if need to recreate the model
if recreate_model:    
    
    # Creating an instance to vectorizer:
    vectorizer = TfidfVectorizer()
    
    # Training the vectorizer:
    X_train_tfidf = vectorizer.fit_transform(X_train)
    
    # Training the classifier:
    clf = LinearSVC()
    clf.fit(X_train_tfidf, y_train)
    
    # Pipelining the vectorizer and the classifier
    text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])
    text_clf.fit(X_train, y_train)
    
    # saving the model to disk
    pickle.dump(text_clf, open(filename, 'wb'))

# If there is no need to recreate the model, just open the file from the disk    
else:
    # loading the model from disk
    text_clf = pickle.load(open(filename, 'rb'))

In [ ]:
predictions = text_clf.predict(X_test)

Classification report for the SVC Linear model

In [ ]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

        ENFJ       0.84      0.58      0.69       319
        ENFP       0.82      0.78      0.80      1249
        ENTJ       0.90      0.80      0.84       577
        ENTP       0.86      0.83      0.84      2324
        ESFJ       0.83      0.45      0.59        33
        ESFP       0.88      0.48      0.62        75
        ESTJ       0.90      0.84      0.87       105
        ESTP       0.95      0.90      0.92       398
        INFJ       0.81      0.84      0.83      2954
        INFP       0.80      0.82      0.81      2391
        INTJ       0.83      0.87      0.85      4531
        INTP       0.84      0.87      0.86      5033
        ISFJ       0.80      0.61      0.69       132
        ISFP       0.81      0.60      0.69       161
        ISTJ       0.86      0.68      0.76       253
        ISTP       0.89      0.79      0.84       679

    accuracy                           0.84     21214
   macro avg       0.85   

Accuracy for the SVC Linear model

In [ ]:
print(f"Overall accuracy of the model: {round(metrics.accuracy_score(y_test, predictions),2)}")

Overall accuracy of the model: 0.84


Example execution for SVC Linear model

In [ ]:
str(text_clf.predict(["Hello world"])[0])

'INFP'

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

class Lemmatizer(object):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
    def __call__(self, sentence):
        return [self.lemmatizer.lemmatize(word) for word in sentence.split() if len(word)>2]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

filename2 = '/content/drive/MyDrive/Colab Notebooks/mbti_svm_v1.sav'
model_cat = pickle.load(open(filename2, 'rb'))

target_encoder = LabelEncoder()

tokenizer = RegexpTokenizer(r'\w+')

vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    tokenizer=lambda text: [WordNetLemmatizer().lemmatize(token.lower()) for token in tokenizer.tokenize(text)],
    lowercase=True
)

vectorizer.fit(X_train)

try:
    X_test = vectorizer.transform(X_test).toarray()
    y_test = target_encoder.fit_transform(y_test)
except Exception as e:
    print(f"Error during vectorization: {e}")
    print(f"Problematic text: {X_test}")




/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.



**Catboost model**

In [ ]:
pip install catboost


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from tqdm import tqdm
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from imblearn.over_sampling import SMOTE
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/drive')
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MBTI 500.csv')
data.head()

/usr/local/lib/python3.9/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning:

Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,posts,type
0,know intj tool use interaction people excuse a...,INTJ
1,rap music ehh opp yeah know valid well know fa...,INTJ
2,preferably p hd low except wew lad video p min...,INTJ
3,drink like wish could drink red wine give head...,INTJ
4,space program ah bad deal meing freelance max ...,INTJ


In [ ]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=42,stratify=data.type)

In [ ]:
def clear_text(data):
    data_length=[]
    lemmatizer=WordNetLemmatizer()
    cleaned_text=[]
    for sentence in tqdm(data.posts):
        sentence=sentence.lower()
        
#         removing links from text data
        sentence=re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',sentence)
    
#         removing other symbols
        sentence=re.sub('[^0-9a-z]',' ',sentence)
    
        
        data_length.append(len(sentence.split()))
        cleaned_text.append(sentence)
    return cleaned_text,data_length

In [ ]:
train_data.posts,train_length=clear_text(train_data)

100%|██████████| 84853/84853 [00:44<00:00, 1904.18it/s]


In [ ]:
test_data.posts,test_length=clear_text(test_data)

100%|██████████| 21214/21214 [00:06<00:00, 3344.97it/s]


In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer


lemmatizer = WordNetLemmatizer()
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', tokenizer=lemmatizer.lemmatize)
vectorizer.fit(train_data.posts)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


TfidfVectorizer(max_features=5000, stop_words='english',
                tokenizer=<bound method WordNetLemmatizer.lemmatize of <WordNetLemmatizer>>)

In [ ]:
train_post=vectorizer.transform(train_data.posts).toarray()
test_post=vectorizer.transform(test_data.posts).toarray()

In [ ]:
train_post.shape

(84853, 25)

In [ ]:
target_encoder=LabelEncoder()
train_target=target_encoder.fit_transform(train_data.type)
test_target=target_encoder.fit_transform(test_data.type)

In [ ]:

from catboost import CatBoostClassifier

model_cat = CatBoostClassifier(task_type="CPU")
model_cat.fit(train_post, train_target)


Learning rate set to 0.09924
0:	learn: 2.5939064	total: 682ms	remaining: 11m 21s
1:	learn: 2.4735150	total: 1.26s	remaining: 10m 31s
2:	learn: 2.3829625	total: 1.83s	remaining: 10m 8s
3:	learn: 2.3154180	total: 2.42s	remaining: 10m 2s
4:	learn: 2.2586930	total: 2.97s	remaining: 9m 51s
5:	learn: 2.2114592	total: 3.68s	remaining: 10m 9s
6:	learn: 2.1725209	total: 4.78s	remaining: 11m 17s
7:	learn: 2.1394359	total: 5.86s	remaining: 12m 6s
8:	learn: 2.1093772	total: 6.88s	remaining: 12m 37s
9:	learn: 2.0850492	total: 7.92s	remaining: 13m 3s
10:	learn: 2.0625815	total: 8.6s	remaining: 12m 53s
11:	learn: 2.0434653	total: 9.15s	remaining: 12m 33s
12:	learn: 2.0267423	total: 9.71s	remaining: 12m 17s
13:	learn: 2.0119402	total: 10.3s	remaining: 12m 4s
14:	learn: 1.9992125	total: 10.8s	remaining: 11m 51s
15:	learn: 1.9873229	total: 11.4s	remaining: 11m 40s
16:	learn: 1.9761193	total: 11.9s	remaining: 11m 30s
17:	learn: 1.9665310	total: 12.5s	remaining: 11m 22s
18:	learn: 1.9579039	total: 13s	rem

In [ ]:
print(accuracy_score(y_test,model_cat.predict(X_test)))

0.23724898651833695


For connecting to Anvil

In [ ]:
import anvil.server

anvil.server.connect("YFVBMK7CHYRL4A2NRUCXDR3S-4QODO5CLYDAYIUP6")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [ ]:
@anvil.server.callable
def SVC(txt):
    print(txt)
    type=text_clf.predict([txt])
    return str(type[0])
@anvil.server.callable
def CAT(txt):
  text=vectorizer.transform([txt]).toarray()
  ans=model_cat.predict(text)
  return str(TYPES[ans[0][0]])